In [1]:
# Import required basic libraries
import sqlite3
import pandas as pd

# 1. Establish SQLite database connection (keep open throughout, close only at the end)
con = sqlite3.connect("my_data1.db")

# 2. Read CSV data and write to database table
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

# 3. Create cursor object (reuse for all SQL executions)
cur = con.cursor()

# ========== Task 1: Display names of unique launch sites ==========
print("=== Task 1: Names of Unique Launch Sites ===")
sql1 = 'SELECT DISTINCT "Launch_Site" FROM SPACEXTBL WHERE "Launch_Site" IS NOT NULL;'
cur.execute(sql1)
for site in cur.fetchall():
    print("- " + site[0])

# ========== Task 2: Display 5 records where launch sites begin with 'CCA' ==========
print("\n=== Task 2: 5 Records with Launch Sites Starting with 'CCA' ===")
sql2 = 'SELECT * FROM SPACEXTBL WHERE "Launch_Site" LIKE \'CCA%\' LIMIT 5;'
cur.execute(sql2)
col_names = [desc[0] for desc in cur.description]
print("Column Names: ", " | ".join(col_names))
print("-" * 120)
for record in cur.fetchall():
    print(" | ".join(str(item) for item in record))

# ========== Task 3: Display total payload mass carried by boosters launched by NASA (CRS) ==========
print("\n=== Task 3: Total Payload Mass by NASA (CRS) Boosters ===")
sql3 = 'SELECT SUM("PAYLOAD_MASS__KG_") AS Total_Payload_Mass FROM SPACEXTBL WHERE "Customer" = \'NASA (CRS)\';'
cur.execute(sql3)
result = cur.fetchone()
print(f"Total Payload Mass (KG): {result[0]}")

# ========== Task 4: Display average payload mass carried by booster version F9 v1.1 ==========
print("\n=== Task 4: Average Payload Mass of Booster Version F9 v1.1 ===")
sql4 = 'SELECT AVG("PAYLOAD_MASS__KG_") AS Avg_Payload_Mass FROM SPACEXTBL WHERE "Booster_Version" = \'F9 v1.1\';'
cur.execute(sql4)
result = cur.fetchone()
print(f"Average Payload Mass (KG): {round(result[0], 2)}")  # Keep 2 decimal places

# ========== Task 5: List the date when the first successful landing outcome in ground pad was achieved ==========
print("\n=== Task 5: First Successful Ground Pad Landing Date ===")
sql5 = 'SELECT MIN("Date") AS First_Successful_Ground_Pad_Landing FROM SPACEXTBL WHERE "Landing_Outcome" = \'Success (ground pad)\';'
cur.execute(sql5)
result = cur.fetchone()
print(f"First Successful Ground Pad Landing Date: {result[0]}")

# ========== Task 6: List booster names with success in drone ship and payload mass 4000-6000 KG ==========
print("\n=== Task 6: Boosters with Successful Drone Ship Landing (4000 < Payload < 6000 KG) ===")
sql6 = 'SELECT DISTINCT "Booster_Version" FROM SPACEXTBL WHERE "Landing_Outcome" = \'Success (drone ship)\' AND "PAYLOAD_MASS__KG_" > 4000 AND "PAYLOAD_MASS__KG_" < 6000;'
cur.execute(sql6)
for booster in cur.fetchall():
    print("- " + booster[0])

# ========== Task 7: List total number of successful and failure mission outcomes ==========
print("\n=== Task 7: Total Count of Successful/Failed Mission Outcomes ===")
sql7 = 'SELECT "Mission_Outcome", COUNT(*) AS Count FROM SPACEXTBL GROUP BY "Mission_Outcome";'
cur.execute(sql7)
for outcome, count in cur.fetchall():
    print(f"{outcome}: {count} times")

# ========== Task 8: List booster versions with maximum payload mass (using subquery) ==========
print("\n=== Task 8: Booster Versions with Maximum Payload Mass ===")
sql8 = 'SELECT DISTINCT "Booster_Version" FROM SPACEXTBL WHERE "PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTBL);'
cur.execute(sql8)
for booster in cur.fetchall():
    print("- " + booster[0])

# ========== Task 9: 2015 monthly records - failure landing outcomes in drone ship ==========
print("\n=== Task 9: 2015 Monthly Failed Drone Ship Landing Records ===")
sql9 = '''
SELECT substr("Date", 6, 2) AS Month, "Booster_Version", "Launch_Site" 
FROM SPACEXTBL 
WHERE substr("Date", 0, 5) = '2015' AND "Landing_Outcome" = 'Failure (drone ship)';
'''
cur.execute(sql9)
# Print header and results
print("Month | Booster Version | Launch Site")
print("-" * 80)
for month, booster, site in cur.fetchall():
    print(f"{month}   | {booster} | {site}")

# ========== Task 10: Rank landing outcome counts (2010-06-04 to 2017-03-20) in descending order ==========
print("\n=== Task 10: Ranked Landing Outcome Counts (2010-06-04 to 2017-03-20) ===")
sql10 = '''
SELECT "Landing_Outcome", COUNT(*) AS Count 
FROM SPACEXTBL 
WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20' 
GROUP BY "Landing_Outcome" 
ORDER BY Count DESC;
'''
cur.execute(sql10)
print("Landing Outcome | Count")
print("-" * 40)
for outcome, count in cur.fetchall():
    print(f"{outcome} | {count}")

# Close database connection at last
con.close()

=== Task 1: Names of Unique Launch Sites ===
- CCAFS LC-40
- VAFB SLC-4E
- KSC LC-39A
- CCAFS SLC-40

=== Task 2: 5 Records with Launch Sites Starting with 'CCA' ===
Column Names:  Date | Time (UTC) | Booster_Version | Launch_Site | Payload | PAYLOAD_MASS__KG_ | Orbit | Customer | Mission_Outcome | Landing_Outcome
------------------------------------------------------------------------------------------------------------------------
2010-06-04 | 18:45:00 | F9 v1.0  B0003 | CCAFS LC-40 | Dragon Spacecraft Qualification Unit | 0 | LEO | SpaceX | Success | Failure (parachute)
2010-12-08 | 15:43:00 | F9 v1.0  B0004 | CCAFS LC-40 | Dragon demo flight C1, two CubeSats, barrel of Brouere cheese | 0 | LEO (ISS) | NASA (COTS) NRO | Success | Failure (parachute)
2012-05-22 | 7:44:00 | F9 v1.0  B0005 | CCAFS LC-40 | Dragon demo flight C2 | 525 | LEO (ISS) | NASA (COTS) | Success | No attempt
2012-10-08 | 0:35:00 | F9 v1.0  B0006 | CCAFS LC-40 | SpaceX CRS-1 | 500 | LEO (ISS) | NASA (CRS) | Succes